In [1]:
from ipymarkup import show_dep_ascii_markup, show_dep_markup, show_span_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Syntax
from yargy import Parser

In [2]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import yake
from laserembeddings import Laser

In [30]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
syntax = Syntax.load('slovnet_syntax_news_v1.tar')
syntax.navec(navec)

Syntax(
    infer=SyntaxInfer(
        model=Syntax(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                    

In [10]:
!python -m dostoevsky download fasttext-social-network-model

In [10]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [19]:
language = "ru"
max_ngram_size = 4
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

In [19]:
kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

In [24]:
!python -m laserembeddings download-models

In [24]:
laser = Laser()

In [4]:
#text = "За последние 10 лет, во втором десятилетии XXI века, рынок стартапов и венчурного инвестирования для многих незаметно, но кардинально изменился. Парадигмой пионеров интернета было создать проект, который изменит интернет к лучшему — «сделать новый Google» (новый Facebook, новый YouTube, новый айфон и так далее). Новой парадигмой стартапера, задумывающегося о своём вкладе в интернет, вступившей в силу в последнее десятилетие, стало создать проект, который будет куплен «Гуглом», «Фейсбуком», Apple (или «Яндексом» или Mail.ru в случае Рунета). Об изменении мира или хотя бы интернета речи уже не идёт — мир уже изменился, а в процессе был открыт, изучен и поделён. Все точки входа и торговые пути под контролем выросших за первые два десятилетия XXI века империй, и новоприбывающим колонистам ничего не остаётся, кроме как выбирать, флагу какой из них присягнуть. Для многих людей, включая автора этих строк, заставших нынешние интернет-империи «в коротких штанишках», когда мы дышали с ними одним воздухом интернет-вольницы, а «Гугл» ещё верил в свой девиз Don't be evil, произошедшее преображение оказалось неприятным, болезненным и неожиданным поворотом. И — отрезвляющим. Но, на самом деле, за последние 10 лет не произошло совершенно ничего нового."
text = "Минтранс предложил оснастить автомобили сервисов такси устройствами для контроля сонливости водителей. Об этом говорится в проекте федерального закона «О такси», размещённого на сайте проектов нормативно-правовых актов. Законопроект предусматривает, чтобы кузов такси окрашивали по схеме из квадратов контрастных цветов в шахматном порядке. При этом цветовая гамма должна «соответствовать стандартам региона». На крыше автомобиля необходимо размещать опознавательный фонарь оранжевого цвета. Если проект примут, он вступит в силу 1 сентября 2022 года. Действующие ПДД разрешают водителю находиться за рулём суммарно не более десяти часов. Контролируют это с помощью тахографов. Штраф для нарушителей — до 2,5 тысяч рублей."

In [5]:
chunk = []
sentences = []
for sent in sentenize(text):
    sentence = {}
    sentence['text'] = sent.text
    sentence['span'] = (sent.start, sent.stop)
    sentences.append(sentence)
    tokens = [_.text for _ in tokenize(sent.text)]
    chunk.append(tokens)
chunk[:1]

[['Минтранс',
  'предложил',
  'оснастить',
  'автомобили',
  'сервисов',
  'такси',
  'устройствами',
  'для',
  'контроля',
  'сонливости',
  'водителей',
  '.']]

In [6]:
markup = next(syntax.map(chunk))

In [7]:
# Convert CoNLL-style format to source, target indices
words, deps = [], []
for token in markup.tokens:
    words.append(token.text)
    source = int(token.head_id) - 1
    target = int(token.id) - 1
    if source > 0 and source != target:  # skip root, loops
        deps.append([source, target, token.rel])
show_dep_ascii_markup(words, deps)

      ┌► Минтранс     nsubj
┌───┌─└─ предложил    
│ ┌─└►┌─ оснастить    xcomp
│ │ ┌─└► автомобили   obj
│ │ └►┌─ сервисов     nmod
│ │   └► такси        nmod
│ └►┌─── устройствами iobj
│   │ ┌► для          case
│ ┌─└►└─ контроля     nmod
│ │   ┌► сонливости   amod
│ └──►└─ водителей    nmod
└──────► .            punct


In [8]:
show_dep_markup(words, deps)

In [12]:
sentences

[{'text': 'Минтранс предложил оснастить автомобили сервисов такси устройствами для контроля сонливости водителей.',
  'span': (0, 102)},
 {'text': 'Об этом говорится в проекте федерального закона «О такси», размещённого на сайте проектов нормативно-правовых актов.',
  'span': (103, 219)},
 {'text': 'Законопроект предусматривает, чтобы кузов такси окрашивали по схеме из квадратов контрастных цветов в шахматном порядке.',
  'span': (220, 340)},
 {'text': 'При этом цветовая гамма должна «соответствовать стандартам региона».',
  'span': (341, 409)},
 {'text': 'На крыше автомобиля необходимо размещать опознавательный фонарь оранжевого цвета.',
  'span': (410, 491)},
 {'text': 'Если проект примут, он вступит в силу 1 сентября 2022 года.',
  'span': (492, 551)},
 {'text': 'Действующие ПДД разрешают водителю находиться за рулём суммарно не более десяти часов.',
  'span': (552, 638)},
 {'text': 'Контролируют это с помощью тахографов.', 'span': (639, 677)},
 {'text': 'Штраф для нарушителей — до 

In [13]:
sent_texts = []
for s in sentences:
    sent_texts.append(s['text'])

In [14]:
sent_texts

['Минтранс предложил оснастить автомобили сервисов такси устройствами для контроля сонливости водителей.',
 'Об этом говорится в проекте федерального закона «О такси», размещённого на сайте проектов нормативно-правовых актов.',
 'Законопроект предусматривает, чтобы кузов такси окрашивали по схеме из квадратов контрастных цветов в шахматном порядке.',
 'При этом цветовая гамма должна «соответствовать стандартам региона».',
 'На крыше автомобиля необходимо размещать опознавательный фонарь оранжевого цвета.',
 'Если проект примут, он вступит в силу 1 сентября 2022 года.',
 'Действующие ПДД разрешают водителю находиться за рулём суммарно не более десяти часов.',
 'Контролируют это с помощью тахографов.',
 'Штраф для нарушителей — до 2,5 тысяч рублей.']

In [15]:
results = model.predict(sent_texts)

In [16]:
delta = 0.001
spans = []
for sentence, sentiment in zip(sentences, results):
    print(sentence['text'], '\n', sentiment)
    main_sentiment = ''
    if sentiment['negative'] > sentiment['positive']:
        main_sentiment = 'Negative'
    if sentiment['positive'] > sentiment['negative']:
        main_sentiment = 'Positive'
    if abs(sentiment['positive'] - sentiment['negative']) < delta:
        main_sentiment = 'Neutral'
    if main_sentiment != '':
        spans.append(sentence['span'] + (main_sentiment,))

Минтранс предложил оснастить автомобили сервисов такси устройствами для контроля сонливости водителей. 
 {'neutral': 0.9219318628311157, 'negative': 0.1824355274438858, 'skip': 0.10375863313674927, 'positive': 0.005394937004894018, 'speech': 0.003085370408371091}
Об этом говорится в проекте федерального закона «О такси», размещённого на сайте проектов нормативно-правовых актов. 
 {'neutral': 0.9740526676177979, 'negative': 0.0980893224477768, 'skip': 0.031153826043009758, 'positive': 0.00942259095609188, 'speech': 0.0020607432816177607}
Законопроект предусматривает, чтобы кузов такси окрашивали по схеме из квадратов контрастных цветов в шахматном порядке. 
 {'neutral': 0.8840492963790894, 'negative': 0.06561483442783356, 'skip': 0.04469086229801178, 'positive': 0.03309597820043564, 'speech': 0.008325778879225254}
При этом цветовая гамма должна «соответствовать стандартам региона». 
 {'neutral': 0.9954004287719727, 'skip': 0.048867784440517426, 'positive': 0.007355827372521162, 'negativ

In [17]:
spans

[(0, 102, 'Negative'),
 (103, 219, 'Negative'),
 (220, 340, 'Negative'),
 (341, 409, 'Positive'),
 (410, 491, 'Neutral'),
 (492, 551, 'Negative'),
 (552, 638, 'Negative'),
 (639, 677, 'Negative'),
 (678, 722, 'Negative')]

In [18]:
show_span_box_markup(text, spans, palette=palette(RED, GREEN, BLUE))

In [21]:
keywords = kw_extractor.extract_keywords(text)
keywords.sort(key = lambda t: t[1])
for kw in keywords :
    print(kw)

('Минтранс предложил оснастить автомобили', 0.002272028513374053)
('предложил оснастить автомобили сервисов', 0.0033777466038870407)
('Минтранс предложил оснастить', 0.009046426423600748)
('контроля сонливости водителей', 0.009046426423600748)
('оснастить автомобили сервисов такси', 0.009242041154937508)
('автомобили сервисов такси устройствами', 0.009242041154937508)
('предложил оснастить автомобили', 0.013342110513698253)
('оснастить автомобили сервисов', 0.013342110513698253)
('устройствами для контроля сонливости', 0.013342110513698253)
('автомобили сервисов такси', 0.03502939240057983)
('сервисов такси устройствами', 0.03502939240057983)
('такси устройствами для контроля', 0.03502939240057983)
('Минтранс предложил', 0.03702887296351132)
('сонливости водителей', 0.03702887296351132)
('предложил оснастить', 0.054012883016816136)
('оснастить автомобили', 0.054012883016816136)
('автомобили сервисов', 0.054012883016816136)
('устройствами для контроля', 0.054012883016816136)
('контроля 

In [22]:
limit = 0.01

for (text, value) in keywords :
    if (value < limit) :
        print(text, ' : ', value)

Минтранс предложил оснастить автомобили  :  0.002272028513374053
предложил оснастить автомобили сервисов  :  0.0033777466038870407
Минтранс предложил оснастить  :  0.009046426423600748
контроля сонливости водителей  :  0.009046426423600748
оснастить автомобили сервисов такси  :  0.009242041154937508
автомобили сервисов такси устройствами  :  0.009242041154937508


In [26]:
embeddings = laser.embed_sentences(sent_texts, lang='en')

In [27]:
print(embeddings)

[[ 6.7700457e-04 -1.4364591e-05 -2.5171307e-03 ...  5.9748804e-03
   7.1614794e-03  3.7041806e-02]
 [ 3.8949952e-03 -2.0896041e-05 -1.6322421e-03 ... -1.7432461e-03
   5.9528567e-04  3.7938036e-02]
 [ 1.0860672e-02 -8.1053688e-05 -1.0471136e-03 ...  3.2550457e-03
   9.0221642e-03  3.2278448e-02]
 ...
 [-4.5993508e-04  1.8560444e-05 -4.8720356e-04 ...  1.8343654e-02
   4.8426827e-03  1.3497765e-02]
 [ 6.5153367e-03 -7.7432906e-06  3.1010910e-05 ...  1.3117737e-02
   6.2167766e-03  2.1801263e-02]
 [ 8.4446510e-03 -6.5196423e-06  2.5322983e-02 ...  1.1143272e-02
   1.7489810e-02  1.9111268e-02]]
